In [3]:
import requests 
import mysql.connector 
import pandas as pd
import json
import re

cnx = mysql.connector.connect(
    host="localhost",
    user="root",
    password="AlumnaAdalab"   # <-- cambia esto
)
cur = cnx.cursor()

cur.execute(f"CREATE DATABASE IF NOT EXISTS musicdata_studiocode")
cur.execute(f"USE musicdata_studiocode")

print("Conectado a MySQL.")

Conectado a MySQL.


In [4]:
cur.execute("SET FOREIGN_KEY_CHECKS = 0")

tablas = [
    "canciones_artistas",
    "canciones",
    "artistas_similares",
    "similares",
    "generos_musicales",
    "artistas"
]

for t in tablas:
    cur.execute(f"DROP TABLE IF EXISTS {t}")

cur.execute("SET FOREIGN_KEY_CHECKS = 1")
cnx.commit()

print("Tablas eliminadas.")

Tablas eliminadas.


In [5]:
cur.execute("""
CREATE TABLE artistas (
  id_artista INT AUTO_INCREMENT PRIMARY KEY,
  nombre VARCHAR(255) NOT NULL UNIQUE,
  numero_reproducciones BIGINT NULL,
  numero_oyentes BIGINT NULL
) ENGINE=InnoDB;
""")
cnx.commit()

print("Tabla artistas creada.")

Tabla artistas creada.


In [6]:
cur.execute("""
CREATE TABLE generos_musicales (
  id_genero INT AUTO_INCREMENT PRIMARY KEY,
  nombre VARCHAR(50) NOT NULL UNIQUE
) ENGINE=InnoDB;
""")
cnx.commit()

print("Tabla generos_musicales creada.")

Tabla generos_musicales creada.


In [7]:
cur.execute("""
CREATE TABLE similares (
  id_similar INT AUTO_INCREMENT PRIMARY KEY,
  nombre_similar VARCHAR(255) NOT NULL UNIQUE
) ENGINE=InnoDB;
""")
cnx.commit()

print("Tabla similares creada.")

Tabla similares creada.


In [8]:
cur.execute("""
CREATE TABLE artistas_similares (
  id_artista INT NOT NULL,
  id_similar INT NOT NULL,
  PRIMARY KEY (id_artista, id_similar),
  FOREIGN KEY (id_artista) REFERENCES artistas(id_artista)
    ON DELETE RESTRICT ON UPDATE CASCADE,
  FOREIGN KEY (id_similar) REFERENCES similares(id_similar)
    ON DELETE RESTRICT ON UPDATE CASCADE
) ENGINE=InnoDB;
""")
cnx.commit()

print("Tabla artistas_similares creada.")

Tabla artistas_similares creada.


In [9]:
cur.execute("""
CREATE TABLE canciones (
  id_cancion INT AUTO_INCREMENT PRIMARY KEY,
  titulo_cancion TEXT NOT NULL,
  id_genero INT NOT NULL,
  anio_lanzamiento INT NULL,
  FOREIGN KEY (id_genero) REFERENCES generos_musicales(id_genero)
    ON DELETE RESTRICT ON UPDATE CASCADE
) ENGINE=InnoDB;
""")

cur.execute("""
CREATE TABLE canciones_artistas (
  id_cancion INT NOT NULL,
  id_artista INT NOT NULL,
  PRIMARY KEY (id_cancion, id_artista),
  FOREIGN KEY (id_cancion) REFERENCES canciones(id_cancion)
    ON DELETE CASCADE ON UPDATE CASCADE,
  FOREIGN KEY (id_artista) REFERENCES artistas(id_artista)
    ON DELETE RESTRICT ON UPDATE CASCADE
) ENGINE=InnoDB;
""")

cnx.commit()
print("Tablas canciones y canciones_artistas creadas.")

Tablas canciones y canciones_artistas creadas.


In [10]:
with open("info_artistas.json", "r", encoding="utf-8") as f:
    info_artistas = json.load(f)

with open("datos_spotify.json", "r", encoding="utf-8") as f:
    datos_spotify = json.load(f)

print("info_artistas:", len(info_artistas))
print("datos_spotify:", len(datos_spotify))

info_artistas: 3827
datos_spotify: 61617


In [11]:
for a in info_artistas:
    nombre = a.get("name")
    if not nombre:
        continue

    playcount = a.get("playcount")
    listeners = a.get("listeners")

    try:
        playcount = int(playcount) if playcount is not None else None
    except:
        playcount = None

    try:
        listeners = int(listeners) if listeners is not None else None
    except:
        listeners = None

    cur.execute(
        """
        INSERT INTO artistas (nombre, numero_reproducciones, numero_oyentes)
        VALUES (%s, %s, %s)
        ON DUPLICATE KEY UPDATE
          numero_reproducciones = VALUES(numero_reproducciones),
          numero_oyentes = VALUES(numero_oyentes)
        """,
        (nombre, playcount, listeners)
    )

cnx.commit()
print("Artistas insertados.")

Artistas insertados.


In [12]:
# 1) Insertar similares únicos
similares_unicos = set()
for a in info_artistas:
    for s in a.get("similares", []):
        if s:
            similares_unicos.add(s)

for s in similares_unicos:
    cur.execute("INSERT IGNORE INTO similares (nombre_similar) VALUES (%s)", (s,))

cnx.commit()

# 2) Cargar diccionarios nombre -> id
cur.execute("SELECT id_artista, nombre FROM artistas")
dic_artistas = {nombre: i for (i, nombre) in cur.fetchall()}

cur.execute("SELECT id_similar, nombre_similar FROM similares")
dic_similares = {nombre: i for (i, nombre) in cur.fetchall()}

# 3) Insertar relaciones
for a in info_artistas:
    nombre_artista = a.get("name")
    if not nombre_artista or nombre_artista not in dic_artistas:
        continue

    id_artista = dic_artistas[nombre_artista]

    for s in a.get("similares", []):
        if s in dic_similares:
            cur.execute(
                "INSERT IGNORE INTO artistas_similares (id_artista, id_similar) VALUES (%s, %s)",
                (id_artista, dic_similares[s])
            )

cnx.commit()
print("Similares y relaciones artista-similar insertadas.")

Similares y relaciones artista-similar insertadas.


In [13]:
cur.execute("DELETE FROM generos_musicales")
cnx.commit()

generos = ["reggaeton", "hip-hop", "rock", "indie", "pop"]

cur.executemany(
    "INSERT INTO generos_musicales (nombre) VALUES (%s)",
    [(g,) for g in generos]
)
cnx.commit()

# Crear diccionario nombre -> id, leyendo en orden
cur.execute("SELECT id_genero, nombre FROM generos_musicales ORDER BY id_genero ASC")
dic_generos = {nombre: i for (i, nombre) in cur.fetchall()}

print("Géneros insertados")
print(dic_generos)

Géneros insertados
{'reggaeton': 1, 'hip-hop': 2, 'rock': 3, 'indie': 4, 'pop': 5}


In [14]:
def normalizar_titulo(titulo):
    titulo = (titulo or "").lower()
    titulo = re.sub(r"[^0-9a-záéíóúñü]+", " ", titulo)
    return re.sub(r"\s+", " ", titulo).strip()

def extraer_anio(valor):
    m = re.search(r"(19\d{2}|20\d{2})", str(valor)) if valor is not None else None
    return int(m.group(1)) if m else None


anio_minimo, anio_maximo = 2021, 2025

cur.execute("SELECT id_artista, nombre FROM artistas")
dic_artistas = {nombre: i for (i, nombre) in cur.fetchall()}

# Mapa interno (solo memoria): track_id -> id_cancion
track_to_idcancion = {}

canciones_creadas = relaciones_creadas = filtradas_por_anio = 0

for item in datos_spotify:
    track_id = item.get("id")                 # solo interno
    artista = item.get("artist_name")         # string en tu JSON
    titulo = item.get("track_name")
    genero = item.get("genre")
    anio = extraer_anio(item.get("year"))

    if anio is None or anio < 2021 or anio > 2025:
        filtradas_por_anio += 1
        continue

    if not track_id or not artista or not titulo or not genero:
        continue

    id_artista = dic_artistas.get(artista)
    id_genero = dic_generos.get(genero)
    if not id_artista or not id_genero:
        continue

    # 1) Si ya vimos este track_id en esta ejecución -> reutiliza id_cancion
    id_cancion = track_to_idcancion.get(track_id)

    # 2) Si no, intenta reutilizar una canción existente (best-effort) o crea una nueva
    if id_cancion is None:
        cur.execute("""
            SELECT id_cancion
            FROM canciones
            WHERE titulo_cancion = %s
              AND anio_lanzamiento = %s
              AND id_genero = %s
            LIMIT 1
        """, (titulo, anio, id_genero))
        fila = cur.fetchone()

        if fila:
            id_cancion = fila[0]
        else:
            cur.execute("""
                INSERT INTO canciones (titulo_cancion, id_genero, anio_lanzamiento)
                VALUES (%s, %s, %s)
            """, (titulo, id_genero, anio))
            id_cancion = cur.lastrowid
            canciones_creadas += 1

        # Guardar el mapeo interno para que cualquier otro artista use el mismo id
        track_to_idcancion[track_id] = id_cancion

    # 3) Relación canción-artista
    cur.execute("""
        INSERT IGNORE INTO canciones_artistas (id_cancion, id_artista)
        VALUES (%s, %s)
    """, (id_cancion, id_artista))
    relaciones_creadas += 1

cnx.commit()

print("Filtro aplicado:", anio_minimo, "hasta", anio_maximo)
print("Filtradas por año:", filtradas_por_anio)
print("Canciones nuevas creadas:", canciones_creadas)
print("Relaciones creadas:", relaciones_creadas)

Filtro aplicado: 2021 hasta 2025
Filtradas por año: 49449
Canciones nuevas creadas: 5198
Relaciones creadas: 7205


In [15]:
if cnx is not None and cnx.is_connected():
    cnx.close() # Cerramos conexión para la realización de la siguiente consulta

## Consultas con Python-MySQL

In [16]:
cnx = mysql.connector.connect(
    user='root',
    password='AlumnaAdalab',
    host='127.0.0.1',
    database= 'musicdata_studiocode',
    auth_plugin = 'mysql_native_password'
)

cur = cnx.cursor()

In [17]:
#1. ¿Cuáles son los artistas con más canciones por año?
query = """
SELECT t.anio, t.nombre, t.total
FROM (
    SELECT 
        c.anio_lanzamiento AS anio,
        a.nombre,
        COUNT(*) AS total
    FROM canciones c
    JOIN canciones_artistas ca ON c.id_cancion = ca.id_cancion
    JOIN artistas a ON a.id_artista = ca.id_artista
    GROUP BY c.anio_lanzamiento, a.id_artista
) t
JOIN (
    SELECT 
        anio,
        MAX(total) AS max_total
    FROM (
        SELECT 
            c.anio_lanzamiento AS anio,
            a.id_artista,
            COUNT(*) AS total
        FROM canciones c
        JOIN canciones_artistas ca ON c.id_cancion = ca.id_cancion
        JOIN artistas a ON a.id_artista = ca.id_artista
        GROUP BY c.anio_lanzamiento, a.id_artista
    ) x
    GROUP BY anio
) m
ON t.anio = m.anio AND t.total = m.max_total
ORDER BY t.anio DESC;
"""

cur.execute(query)

print("Artista(s) con más canciones por año:")
for anio, nombre, total in cur.fetchall():
    print(f"Año {anio}: {nombre} - {total} canciones")

Artista(s) con más canciones por año:
Año 2025: Sabrina Carpenter - 15 canciones
Año 2024: The Marías - 15 canciones
Año 2023: Mikel Izal - 15 canciones
Año 2022: Bad Bunny - 21 canciones
Año 2021: Taylor Swift - 13 canciones
Año 2021: Miguel Campello - 13 canciones
Año 2021: Arde Bogotá - 13 canciones
Año 2021: Morat - 13 canciones


In [ ]:
#2. Top 5 por oyentes
query_oyentes = """
SELECT nombre, numero_oyentes
FROM artistas
WHERE numero_oyentes IS NOT NULL
ORDER BY numero_oyentes DESC
LIMIT 5;
"""

cur.execute(query_oyentes)
resultados_oy = cur.fetchall()


#Top 5 por reproducciones
query_reproducciones = """
SELECT nombre, numero_reproducciones
FROM artistas
WHERE numero_reproducciones IS NOT NULL
ORDER BY numero_reproducciones DESC
LIMIT 5;
"""

cur.execute(query_reproducciones)
resultados_rep = cur.fetchall()


print("Top 5 artistas con más oyentes:")
for nombre, oyentes in resultados_oy:
    print(nombre, int(oyentes))

print("\nTop 5 artistas con más reproducciones:")
for nombre, reproducciones in resultados_rep:
    print(nombre, int(reproducciones))

Top 5 artistas con más oyentes:
Coldplay 9006599
Radiohead 8133875
Kanye West 7852276
Eminem 7715162
Lady Gaga 7617852

Top 5 artistas con más reproducciones:
Taylor Swift 3596382104
BTS 2914181072
Lana Del Rey 1540331320
Kanye West 1482955709
Radiohead 1341210663


In [19]:
# 3. ¿Qué artista tiene más generos? 
query = """
SELECT a.nombre, COUNT(DISTINCT c.id_genero) AS total_generos
FROM artistas a
JOIN canciones_artistas ca ON a.id_artista = ca.id_artista
JOIN canciones c ON c.id_cancion = ca.id_cancion
GROUP BY a.id_artista, a.nombre
ORDER BY total_generos DESC
LIMIT 1;
"""

cur.execute(query)
nombre, total = cur.fetchone()

print("Artista con más géneros:")
print(nombre, total)

Artista con más géneros:
Miguel Campello 3


In [ ]:
#4. ¿Qué artista tiene más colaboraciones?

query = """
WITH por_cancion AS (
  SELECT id_cancion, COUNT(*) AS n_artistas
  FROM canciones_artistas
  GROUP BY id_cancion
  HAVING COUNT(*) > 1
),
colabs_por_artista AS (
  SELECT ca.id_artista, SUM(pc.n_artistas - 1) AS total_colaboraciones
  FROM canciones_artistas ca
  JOIN por_cancion pc ON pc.id_cancion = ca.id_cancion
  GROUP BY ca.id_artista
)
SELECT a.nombre, cpa.total_colaboraciones
FROM colabs_por_artista cpa
JOIN artistas a ON a.id_artista = cpa.id_artista
WHERE cpa.total_colaboraciones = (SELECT MAX(total_colaboraciones) FROM colabs_por_artista);
"""
cur.execute(query)

print("Artista(s) con más colaboraciones:")
for nombre, total in cur.fetchall():
    print(nombre, int(total))

Artista(s) con más colaboraciones:
Duki 36


In [21]:
#5. ¿En qué año se lanzaron más canciones?
query = """
SELECT anio_lanzamiento, COUNT(*) AS total_canciones
FROM canciones
GROUP BY anio_lanzamiento
ORDER BY total_canciones DESC
LIMIT 1;
"""
cur.execute(query)
anio, total = cur.fetchone()
print("Año con más canciones:", anio)
print("Total:", total)

Año con más canciones: 2021
Total: 1202


In [22]:
#6. ¿Cuántos artistas tiene cada género (según popularidad)?
query = """
SELECT 
    g.nombre,
    COUNT(DISTINCT a.id_artista) AS total_artistas,
    AVG(a.numero_oyentes) AS promedio_oyentes
FROM generos_musicales g
JOIN canciones c ON c.id_genero = g.id_genero
JOIN canciones_artistas ca ON ca.id_cancion = c.id_cancion
JOIN artistas a ON a.id_artista = ca.id_artista
GROUP BY g.id_genero, g.nombre
ORDER BY promedio_oyentes DESC;
"""

cur.execute(query)

print("Artistas por género (ordenado por popularidad media):")
for genero, total_artistas, promedio in cur.fetchall():
    print(genero, total_artistas, int(promedio) if promedio else 0)

Artistas por género (ordenado por popularidad media):
pop 537 1367640
rock 591 1313443
reggaeton 48 897801
hip-hop 1154 593806
indie 579 565341


In [ ]:
#6. Canciones por año
query = """
SELECT
    anio_lanzamiento,
    COUNT(*) AS total_canciones
FROM canciones
GROUP BY anio_lanzamiento
ORDER BY total_canciones DESC;
"""

cur.execute(query)

for anio, total in cur.fetchall():
    print(f"{anio}: {total} canciones")

2021: 1202 canciones
2022: 1185 canciones
2023: 1029 canciones
2024: 959 canciones
2025: 823 canciones


In [ ]:
#7. ¿Cuál es el artista similar que más se repite?
query = """
SELECT a2.nombre, COUNT(*) AS repeticiones
FROM artistas_similares s
JOIN artistas a2 ON a2.id_artista = s.id_similar
GROUP BY s.id_similar, a2.nombre
ORDER BY repeticiones DESC
LIMIT 1;
"""
cur.execute(query)
print(cur.fetchone())



('T-Chord', 30)


In [ ]:
#8. Cantidad de artistas por género
query = """
SELECT 
    g.nombre AS genero,
    COUNT(DISTINCT a.id_artista) AS total_artistas
FROM generos_musicales g
JOIN canciones c ON c.id_genero = g.id_genero
JOIN canciones_artistas ca ON ca.id_cancion = c.id_cancion
JOIN artistas a ON a.id_artista = ca.id_artista
GROUP BY g.id_genero, g.nombre
ORDER BY total_artistas DESC;
"""

cur.execute(query)
resultados = cur.fetchall()

print("Cantidad de artistas por género:")
for genero, total in resultados:
    print(f"{genero}: {total}")

Cantidad de artistas por género:
hip-hop: 1154
rock: 591
indie: 579
pop: 537
reggaeton: 48


In [ ]:
#9. Artistas más inactivos
cur = cnx.cursor()
cur.execute("USE musicdata_studiocode;")
query_inactivos = """
    SELECT a.nombre AS Artista, ((MAX(c.anio_lanzamiento) - MIN(c.anio_lanzamiento) + 1) - COUNT(DISTINCT c.anio_lanzamiento)) AS Cantidad_Anios_Inactivos
    FROM artistas a
    JOIN canciones_artistas ca ON a.id_artista = ca.id_artista
    JOIN canciones c ON ca.id_cancion = c.id_cancion
    GROUP BY a.id_artista, a.nombre
    HAVING (MAX(c.anio_lanzamiento) - MIN(c.anio_lanzamiento) + 1) > 1 AND ((MAX(c.anio_lanzamiento) - MIN(c.anio_lanzamiento) + 1) - COUNT(DISTINCT c.anio_lanzamiento)) > 0
    ORDER BY Cantidad_Anios_Inactivos DESC 
    LIMIT 5; """
    
cur.execute(query_inactivos)
resultado_inactivos = cur.fetchall()
print("--- TOP 5 ARTISTAS MÁS INACTIVOS ---")
print(pd.DataFrame(resultado_inactivos, columns=cur.column_names).to_string(index=False))

--- TOP 5 ARTISTAS MÁS INACTIVOS ---
       Artista  Cantidad_Anios_Inactivos
         Yarea                         3
 Justin Bieber                         3
       Volbeat                         3
          Leto                         3
Matt Berninger                         3


In [ ]:
#10. Artistas con mayor diversidad de géneros
cur = cnx.cursor() 
cur.execute("USE musicdata_studiocode;")

query_versatilidad = """
SELECT 
    a.nombre AS Artista, 
    COUNT(DISTINCT c.id_genero) AS Cantidad_Generos,
    GROUP_CONCAT(DISTINCT gm.nombre ORDER BY gm.nombre ASC SEPARATOR ' | ') AS Generos_Explorados
FROM 
    artistas a
JOIN 
    canciones_artistas ca ON a.id_artista = ca.id_artista
JOIN 
    canciones c ON ca.id_cancion = c.id_cancion
JOIN 
    generos_musicales gm ON c.id_genero = gm.id_genero
GROUP BY 
    a.id_artista, a.nombre
ORDER BY 
    Cantidad_Generos DESC 
LIMIT 10;
"""

cur.execute(query_versatilidad)
resultado_versatilidad = cur.fetchall()

print("--- TOP 5 ARTISTAS CON MAYOR DIVERSIDAD DE GÉNEROS ---")
print(pd.DataFrame(resultado_versatilidad, columns=cur.column_names).to_string(index=False))
print("\n")

--- TOP 5 ARTISTAS CON MAYOR DIVERSIDAD DE GÉNEROS ---
        Artista  Cantidad_Generos   Generos_Explorados
          Serko                 3   indie | pop | rock
     Charli xcx                 3   indie | pop | rock
Suki Waterhouse                 3   indie | pop | rock
         Marlon                 3   indie | pop | rock
Miguel Campello                 3   indie | pop | rock
           Maki                 3 hip-hop | pop | rock
    Nil Moliner                 3   indie | pop | rock
       Eyedress                 3   indie | pop | rock
       Måneskin                 3   indie | pop | rock
          sombr                 3   indie | pop | rock




In [ ]:
#11. Cantidad de canciones por género
query = """
SELECT 
    g.nombre AS genero,
    COUNT(c.id_cancion) AS total_canciones
FROM generos_musicales g
JOIN canciones c ON c.id_genero = g.id_genero
GROUP BY g.id_genero, g.nombre
ORDER BY total_canciones DESC;
"""

cur.execute(query)
resultados = cur.fetchall()

print("Cantidad de canciones por género:")
for genero, total in resultados:
    print(f"{genero}: {total}")

Cantidad de canciones por género:
hip-hop: 1700
rock: 1147
pop: 1115
indie: 1080
reggaeton: 156
